In [ ]:
import requests
response = requests.get('https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json', params={'id' :"4e236f34-b981-41c3-8c65-f8c9000b94e7"})

if response.status_code == 200:
    print("Successful connection with API.")
    print('-------------------------------')
    data = response.json()
elif response.status_code == 404:
    print("Unable to reach URL.")
else:
    print("Unable to connect API or retrieve data.")
for record in data:
     print("Gender: {},\n Income: {},\n Married: {},\n".format(record['Gender'] , record['Married'],record['Income']))

In [ ]:
# Import necessary libs
import pandas as pd
from pyspark.sql import SparkSession 
from pyspark.sql import functions as sf
from functools import reduce
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime
import time

from pyspark.conf import SparkConf
from pyspark.context import SparkContext

conf = SparkConf().setAppName("DataExtractionApp").setMaster("local[2]")
sc = SparkSession.builder.config("spark.driver.host", "localhost") \
.config ("spark.sql.execution.arrow.enabled", "true").getOrCreate()


In [ ]:

df_loan = sc.createDataFrame(response.json())
df_loan.show()

In [ ]:
df_loan.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.CDW_SAPP_loan_application") \
.option("user", "root") \
.option("password", "lakshmi") \
.save()